In [9]:
import pandas as pd
import warnings
import datetime
import json
import sys
import numpy as np

In [10]:
warnings.filterwarnings('ignore')

In [11]:
df=pd.read_csv("city_temperature.csv")

In [12]:
df

,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9
...,...,...,...,...,...,...,...,...
2906322,North America,US,Additional Territories,San Juan Puerto Rico,7,27,2013,82.4
2906323,North America,US,Additional Territories,San Juan Puerto Rico,7,28,2013,81.6
2906324,North America,US,Additional Territories,San Juan Puerto Rico,7,29,2013,84.2
2906325,North America,US,Additional Territories,San Juan Puerto Rico,7,30,2013,83.8


In [13]:
df=df[df['City']=='Charleston']
df=df[df['State']=='South Carolina']

In [14]:
df

,Region,Country,State,City,Month,Day,Year,AvgTemperature
2556734,North America,US,South Carolina,Charleston,1,1,1995,52.3
2556735,North America,US,South Carolina,Charleston,1,2,1995,53.5
2556736,North America,US,South Carolina,Charleston,1,3,1995,45.8
2556737,North America,US,South Carolina,Charleston,1,4,1995,39.9
2556738,North America,US,South Carolina,Charleston,1,5,1995,36.4
...,...,...,...,...,...,...,...,...
2565994,North America,US,South Carolina,Charleston,5,9,2020,64.5
2565995,North America,US,South Carolina,Charleston,5,10,2020,60.7
2565996,North America,US,South Carolina,Charleston,5,11,2020,66.9
2565997,North America,US,South Carolina,Charleston,5,12,2020,60.4


In [15]:
df = df[ ['Month',  'Day','Year','AvgTemperature'] ]

In [16]:
df

,Month,Day,Year,AvgTemperature
2556734,1,1,1995,52.3
2556735,1,2,1995,53.5
2556736,1,3,1995,45.8
2556737,1,4,1995,39.9
2556738,1,5,1995,36.4
...,...,...,...,...
2565994,5,9,2020,64.5
2565995,5,10,2020,60.7
2565996,5,11,2020,66.9
2565997,5,12,2020,60.4


In [17]:
df = df.replace(-99.0, None)
df['AvgTemperature'] = (df['AvgTemperature'] - 32) / 1.8  # celcius!!!!
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df = df[ ['Date','AvgTemperature'] ]

In [18]:
df

,Date,AvgTemperature
2556734,1995-01-01,11.277778
2556735,1995-01-02,11.944444
2556736,1995-01-03,7.666667
2556737,1995-01-04,4.388889
2556738,1995-01-05,2.444444
...,...,...
2565994,2020-05-09,18.055556
2565995,2020-05-10,15.944444
2565996,2020-05-11,19.388889
2565997,2020-05-12,15.777778


In [20]:
def linreg(X, Y):
    
    N = len(X)
    Sx = Sy = Sxx = Syy = Sxy = 0.0
    for x, y in zip(X, Y):
        Sx = Sx + x
        Sy = Sy + y
        Sxx = Sxx + x*x
        Syy = Syy + y*y
        Sxy = Sxy + x*y
    det = Sxx * N - Sx * Sx
    return (Sxy * N - Sy * Sx)/det, (Sxx * Sy - Sx * Sxy)/det

In [21]:
df.isnull().sum()

Date               0
AvgTemperature    25
dtype: int64

In [22]:
df = df.bfill()

In [23]:
df.isnull().sum()

Date              0
AvgTemperature    0
dtype: int64

In [24]:
x = df.dropna()['AvgTemperature'].values

In [25]:
x

array([11.27777778, 11.94444444,  7.66666667, ..., 19.38888889,
       15.77777778, 13.77777778])

In [27]:
m,n= linreg(range(len(x)),x) 
df['x'] = range(len(df))# create an index
df['TREND'] = df['x'] * m + n
df['DE-TREND'] = df['AvgTemperature'] - df['TREND']

In [28]:
df

,Date,AvgTemperature,x,TREND,DE-TREND
2556734,1995-01-01,11.277778,0,18.291845,-7.014067
2556735,1995-01-02,11.944444,1,18.291960,-6.347516
2556736,1995-01-03,7.666667,2,18.292076,-10.625409
2556737,1995-01-04,4.388889,3,18.292191,-13.903302
2556738,1995-01-05,2.444444,4,18.292306,-15.847862
...,...,...,...,...,...
2565994,2020-05-09,18.055556,9260,19.360533,-1.304977
2565995,2020-05-10,15.944444,9261,19.360648,-3.416204
2565996,2020-05-11,19.388889,9262,19.360764,0.028125
2565997,2020-05-12,15.777778,9263,19.360879,-3.583101


In [29]:
for i in [1,2,3,4,5,6,7,10,14,28,30,31,60,180,360,365,366]:
    print( i, df['DE-TREND'].autocorr(i) )

1 0.921051767836999
2 0.8352420567613655
3 0.7986521497338683
4 0.7855215304215447
5 0.780447194176237
6 0.7779459554320347
7 0.7725464764745889
10 0.7554887154478886
14 0.7456218698148774
28 0.6714563702094025
30 0.6547507464267892
31 0.6451786142341037
60 0.38523342543735756
180 -0.7469906094124058
360 0.7501514951139161
365 0.7485136241792899
366 0.7455350440698066


In [30]:
for i in [1,2,180,365]:
    df[f'LAG{i}'] = df['DE-TREND'].shift(i)

In [31]:
df

,Date,AvgTemperature,x,TREND,DE-TREND,LAG1,LAG2,LAG180,LAG365
2556734,1995-01-01,11.277778,0,18.291845,-7.014067,NaN,NaN,NaN,NaN
2556735,1995-01-02,11.944444,1,18.291960,-6.347516,-7.014067,NaN,NaN,NaN
2556736,1995-01-03,7.666667,2,18.292076,-10.625409,-6.347516,-7.014067,NaN,NaN
2556737,1995-01-04,4.388889,3,18.292191,-13.903302,-10.625409,-6.347516,NaN,NaN
2556738,1995-01-05,2.444444,4,18.292306,-15.847862,-13.903302,-10.625409,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2565994,2020-05-09,18.055556,9260,19.360533,-1.304977,-2.971528,-2.915858,-7.117537,4.126036
2565995,2020-05-10,15.944444,9261,19.360648,-3.416204,-1.304977,-2.971528,-4.117652,4.737032
2565996,2020-05-11,19.388889,9262,19.360764,0.028125,-3.416204,-1.304977,-15.562212,5.292472
2565997,2020-05-12,15.777778,9263,19.360879,-3.583101,0.028125,-3.416204,-13.951217,3.847912


In [35]:
for r in [3,5,7,30]:
    df[f'MA{r}']=df['DE-TREND'].rolling(r).mean()
    print("MA", r, df[f'MA{r}'].corr(df['DE-TREND']))

MA 3 0.9535841635906224
MA 5 0.9223301231831929
MA 7 0.9081649025888388
MA 30 0.850514958639337


In [36]:
df.dropna()

,Date,AvgTemperature,x,TREND,DE-TREND,LAG1,LAG2,LAG180,LAG365,MA3,MA5,MA7,MA30
2557099,1996-01-01,14.777778,365,18.333969,-3.556191,-5.278298,-10.778183,10.797916,-7.014067,-6.537557,-10.611516,-11.928861,-9.995259
2557100,1996-01-02,15.333333,366,18.334085,-3.000751,-3.556191,-5.278298,10.353356,-6.347516,-3.945080,-7.900520,-10.119453,-9.875004
2557101,1996-01-03,13.777778,367,18.334200,-4.556422,-3.000751,-3.556191,7.853240,-10.625409,-3.704455,-5.433969,-8.659251,-9.904749
2557102,1996-01-04,4.722222,368,18.334315,-13.612093,-4.556422,-3.000751,9.464236,-13.903302,-7.056422,-6.000751,-8.238731,-10.234494
2557103,1996-01-05,5.222222,369,18.334431,-13.112209,-13.612093,-4.556422,9.186343,-15.847862,-10.426908,-7.567533,-7.699164,-10.517943
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2565994,2020-05-09,18.055556,9260,19.360533,-1.304977,-2.971528,-2.915858,-7.117537,4.126036,-2.397454,0.461920,1.473147,0.009659
2565995,2020-05-10,15.944444,9261,19.360648,-3.416204,-1.304977,-2.971528,-4.117652,4.737032,-2.564236,-1.382640,0.663508,-0.140456
2565996,2020-05-11,19.388889,9262,19.360764,0.028125,-3.416204,-1.304977,-15.562212,5.292472,-1.564352,-2.116088,-0.154068,0.063132
2565997,2020-05-12,15.777778,9263,19.360879,-3.583101,0.028125,-3.416204,-13.951217,3.847912,-2.323727,-2.249537,-1.495453,-0.020317


In [37]:
trend=df['TREND']

In [38]:
del df['TREND']
del df['x']
del df['AvgTemperature']

In [45]:
del df['Date']

In [53]:
y=df['DE-TREND']

In [54]:
x=df.drop(columns=['DE-TREND'])

In [52]:
df.isnull().sum()

DE-TREND    0
LAG1        0
LAG2        0
LAG180      0
LAG365      0
MA3         0
MA5         0
MA7         0
MA30        0
dtype: int64

In [51]:
df=df.dropna()

In [55]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=5)
regr.fit(x, y)

RandomForestRegressor(max_depth=5)

In [56]:
print( regr.score(x, y) )

0.9545441521864817
